In [1]:
# !pip install Mastodon.py==1.8.0
# !pip install requests==2.25.1
# !pip install pandas==1.2.4
# !pip install textblob==0.17.1

# Python version 3.8.8

In [22]:
from mastodon import Mastodon, MastodonNotFoundError
import requests
from requests.auth import HTTPBasicAuth
import datetime as dt
import pandas as pd
import re
from io import StringIO
from html.parser import HTMLParser
from textblob import TextBlob

In [ ]:
# Helper function for cleaning toots from HTML tags - may not be necessary when sending toots to CounchDB

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
# Login to ADO

API_KEY = "a2a6e2f96a2c94a04661aacc6e394caa"

url = "https://api.ado.eresearch.unimelb.edu.au/login"
res = requests.post(url, auth=HTTPBasicAuth('apikey', API_KEY))
if res.ok:
    jwt = res.text

In [93]:
# Generate ADO query

text_kws = ['interest rate', ' rba', 'rba decision', "rba's decision", 
            'cash rate', 'interest payment', 'interest repayment', 'interest re-payment', 'repayment of interest', 
            'variable interest', 'fixed interest', 'bank interest', 'rate hike', 
            'mortgage', 'rent payment', 'house rent', 'houserent', 'house payment', 'housing', 
            'inflation', 'cpi index', 'cost of living', 'shrinkflation', 
            'social security', 'job seeker', 'jobseeker', 'youth allowance', 'austudy', 'centrelink', 'centerlink']
look_back_days = 7

today = dt.datetime.now().date()
text_kws = [f'text:"{kw}"' for kw in text_kws]
dates = [f"""date:\"{date.strftime('%Y%m%d')}\"""" for date in pd.date_range(today-dt.timedelta(days=look_back_days), today)]

query = '('+' OR '.join(text_kws) +') AND (' + ' OR '.join(dates) + ')'
#query = '('+' OR '.join(text_kws) +')'
#query

In [94]:
# Retrivie post ids - this takes around 1 min

url = 'https://api.ado.eresearch.unimelb.edu.au/analysis/textsearch/collections/mastodon'

# query parameters set in the dict below
qs_params = { 'query' : query}
headers = {'Authorization': f"Bearer {jwt}"}
res = requests.get(url, headers = headers, params=qs_params)
result = res.json()

while True: #each API request returns 200 ids, need multiple requests in order to get all toots
    headers = {'Authorization': f"Bearer {jwt}", 'x-ado-bookmark':res.headers['x-ado-bookmark']}
    res = requests.get(url, headers = headers, params=qs_params)
    result_2 = res.json()
    if result_2 == []:
        break
    result.extend(result_2)


In [95]:
len(result)

1723

In [97]:
#Retrive toots from Mastodon API

token= "f8-yNpSEGxPcQwwz_GxYtET3VumoGZ6KAQAH6bybICE"

for line in result[:100]: #capping the number of toots to display (for now)
    id = line.split('/')
    try:
        #TO DO: REPLACE THE BELOW AND SEND THE DATA RO COUCHDB
        print(f'\n{id}')
        toot = Mastodon(api_base_url=f'https://{id[0]}', 
                     access_token = token).status(id[2])['content']
        toot_clean = strip_tags(toot)
        print(toot_clean)
        print("Sentiment: ",TextBlob(toot_clean).sentiment.polarity)
    except MastodonNotFoundError: #TO DO: REMOVE THIS EXCEPTION
        print('*** Post not found')
        continue
    except:
        print("Error getting the toot") #TO DO: REMOVE PRINT
        continue


['aus.social', '@augustusbrown', '110361811581441142']
*** Post not found

['mastodon.au', '@Gary', '110341497558444917']
https://www.theguardian.com/australia-news/2023/may/09/federal-budget-2023-welfare-cost-of-living-crisis-relief-plan-jobseeker-youth-allowance-increase-rent-assistance-affordable-housing-australiaJim Chalmers concedes some Australians want the government to do more to help those struggling to pay rent and bills.
Sentiment:  0.5

['mas.to', '@therightarticle', '110350057146303555']
Interest Rate Hikes Are Drowning Us in Debt Today's interest rate hike will force millions of households to borrow just to make ends meet, doing nothing to curb inflation caused by the rich. We need a debt write off to escape the cost of living crisishttps://tribunemag.co.uk/2023/05/interest-rate-hikes-are-drowning-us-in-debt/
Sentiment:  0.375

['mastodon.au', '@Gary', '110341924131740031']
https://www.abc.net.au/news/2023-05-10/budget-2023-centrelink-rent-austudy-jobseeker-changes-expla

About 700,000 households missed mortgage or rent payments last month as the cost of living crisis continued to take its toll, according to a survey https://www.thetimes.co.uk/article/700-000-miss-rent-and-mortgage-payments-in-inflation-crisis-2m2tlg2ps?utm_medium=Social&utm_source=Twitter#Echobox=1683618254-1 #press
Sentiment:  0.0

['aus.social', '@MorpheusB', '110326563968243323']
#Economics #Australia #RBA Interest rate rises feed inflation? Who knew? And so do increasing profits of companies!https://www.abc.net.au/news/2023-05-05/ross-garnaut-rate-hikes-feed-inflation-urges-policy-overhaul/102302152
Sentiment:  0.0

['weirder.earth', '@varve', '110353098030570785']
Error getting the toot

['federated.press', '@OpenForum', '110340723697379305']
A budget for Labor’s battlersA big incentive for Australian doctors to bulk bill, a modest $40 a fortnight rise in JobSeeker and extra rent assistance are highlights of a Labor budget that juggles targeted cost-of-living relief with containin

Grim sign for #Aussie #mortgage holders as rate rises look inevitable.There appeared to be some light at the end of the tunnel for cash-strapped Aussies – but all signs point to a big crash back down to earth. #greatread https://www.news.com.au/finance/economy/interest-rates/grim-sign-for-aussie-mortgage-holders-as-rate-rises-look-inevitable/news-story/a5220c0e3ddeed225e3b2ca1887ad76b
Sentiment:  -0.1511111111111111

['mstdn.ca', '@noahpinion_mirror', '110364682052236999']
Error getting the toot

['mastodon.social', '@onthisday', '110365684141382360']
Today in 2006, 17 years ago: demonstrations for decent housing took place in Spain, which had been called anonymously by email.#OnThisDay
Sentiment:  0.16666666666666666

['mastodon.social', '@ShingoMouse', '110365953580007146']
#CrowdFunding #SaveSocialSecurity #Donations Progressive Turnout Project — Donate via ActBlue https://secure.actblue.com/donate/ms_ptp_fr_q22023_save-social-security-mg-300x?refcode=MS_TEXT_FR_2023.05.13_B2_social

China's inflation hardly has a pulse. That is a problem, writes @Moss_Eco https://trib.al/1pu7PdR via @opinion #press
Sentiment:  -0.2916666666666667

['regenerate.social', '@TraRay', '110364310074880508']
Pt. 4: I have been shaped by my life in Venezuela, & can't help but feel like I am participating in a sacred and ancestral rite of survival - keeping essential practical knowledge alive and passing it along to the next generation.Eating is an act of survival. Eating well can be the difference between life and death during hard times. When markets shut down, products disappear from the shelves, and inflation erases your buying power - how will you truly nourish our body?
Sentiment:  -0.06944444444444445

['mastodon.green', '@SteveJonesnono1', '110349877360068013']
I see the clueless wonders at the Bank of England are at it again.They claim economics is a science whilst treating it like a religion.https://www.theguardian.com/business/2023/may/11/uk-interest-rate-rise-how-it-affects-you